In [1]:
import pandas as pd
import nltk
import spacy
import gensim
import seaborn as sb
from gensim import corpora, models, similarities
from spacy.lang.en import English
from nltk.corpus import wordnet as wn

In [2]:
data=pd.read_csv("DurhamCounty_original_data.csv",)

In [3]:
data.head()

,Unnamed: 0,page_number,word
0,1,1,approved
1,2,1,budget
2,3,1,fy
3,4,2,durham
4,5,2,county


In [4]:
data.drop(data.columns[0], axis=1)

,page_number,word
0,1,approved
1,1,budget
2,1,fy
3,2,durham
4,2,county
5,2,north
6,2,carolina
7,2,fy
8,2,approved
9,2,budget


In [5]:
data.describe()

,Unnamed: 0,page_number
count,97576.00000,97576.000000
mean,48788.50000,161.767474
std,28167.90927,94.198533
min,1.00000,1.000000
25%,24394.75000,78.000000
50%,48788.50000,158.000000
75%,73182.25000,243.000000
max,97576.00000,323.000000


In [6]:
#cleaning 
spacy.load('en')
parser=English()
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.orth_.startswith('http'):
            lda_tokens.append('com')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [7]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\messi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [8]:
def get_lemma(word):
    lemma=wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma

In [9]:
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

In [10]:
#filtering stopwords
nltk.download('stopwords')
stop=set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\messi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
#preparing for lda
def prepare_lda(text):
    tokens=tokenize(text)
    tokens=[token for token in tokens if len(token)>4]
    tokens=[token for token in tokens if token not in stop]
    tokens=[get_lemma(token) for token in tokens]
    return tokens

In [12]:
#creating tokens
import random
text_data=[]
with open('GuilfordCounty_original_data.csv') as f:
    for line in f:
        tokens=prepare_lda(line)
        if random.random()>.99:
            print(tokens)
            text_data.append(tokens)

['31",2,"av']
['291",3,"contest']
['292",3,"theme']
['303",3,"anniversary']
['487",5,"program']
['534",7,"calendar']
['535",7,"amendment']
['686",8,"services']
['688",8,"justice']
['755",9,"support']
['775",9,"issues']
['882",11,"phillips']
['898",11,"honorable']
['1143",11,"the']
['1270",12,"to']
['1282",12,"countys']
['1303",12,"later']
['1367",12,"current']
['1603",12,"charter']
['1725",13,"and']
['1742",13,"islation']
['1758",13,"ba']
['1759",13,"lance']
['1848",13,"respectively']
['1851",13,"three']
['2082",15,"from']
['2112",15,"are']
['2152",15,"care']
['2414",15,"year']
['2522",15,"how']
['2532",15,"have']
['2582",16,"for']
['2704",16,"as']
['2964",17,"for']
['3008",17,"by']
['3045",17,"juvenile']
['3079",17,"s']
['3121",17,"summa']
['3151",17,"s']
['3236",18,"through']
['3301",18,"property']
['3439",18,"ocation']
['3519",18,"also']
['3542",18,"the']
['3704",19,"community']
['3848",19,"andor']
['3948",19,"to']
['4106",20,"budget']
['4184",20,"es']
['4292",20,"after']
['4351",20

['33705",156,"administration']
['33814",157,"maintain']
['33839",157,"chg']
['34026",158,"behavior']
['34027",158,"changes']
['34372",159,"positions']
['34426",159,"in']
['34557",159,"offered']
['34590",160,"december']
['34698",161,"revenue']
['34709",161,"revenue']
['34934",162,"fy']
['34984",162,"gc']
['35008",162,"begin']
['35017",162,"on']
['35030",162,"increased']
['35047",162,"million']
['35105",163,"supplies']
['35193",163,"hazmat']
['35213",163,"assistance']
['35357",163,"communitys']
['35472",163,"the']
['35484",163,"greensboro']
['35523",164,"also']
['35622",164,"reversal']
['35663",164,"property']
['35703",164,"network']
['35732",164,"expenses']
['35947",165,"the']
['36067",165,"plans']
['36213",166,"stretcher']
['36282",166,"cardiac']
['36326",166,"intervention']
['36664",170,"criteria']
['36692",170,"management']
['36742",170,"future']
['36970",170,"by']
['37060",170,"and']
['37490",173,"objectives']
['37517",173,"implement']
['37551",173,"education']
['37568",173,"through

['68888",312,"resulting']
['68934",313,"st']
['69046",314,"departmental']
['69058",314,"and']
['69139",314,"county']
['69255",314,"to']
['69327",314,"to']
['69431",315,"income']
['69582",316,"is']
['69758",316,"to']
['69848",317,"of']
['69886",317,"effect']
['69900",317,"contributed']
['70043",319,"choices']
['70067",319,"typically']
['70164",319,"technology']
['70249",319,"separate']
['70269",319,"for']
['70302",319,"countys']
['70406",321,"fy']
['70414",321,"county']
['70448",321,"training']
['70561",321,"xray']
['70577",321,"paint']
['70582",321,"family']
['70593",321,"freezer']
['70768",323,"interior']
['70830",323,"substance']
['70869",323,"walls']
['70934",324,"exterior']
['70949",324,"building']
['71029",324,"items']
['71184",325,"paving']
['71284",326,"wildlife']
['71339",326,"exterior']
['71357",326,"water']
['71413",327,"circle']
['71480",327,"exterior']
['71589",328,"montileu']
['71633",328,"paving']
['71935",330,"greenway']
['71962",330,"interior']
['72145",331,"foundation'

In [13]:
#creating dictionary
from gensim import corpora
dictionary=corpora.Dictionary(text_data)
corpus= [dictionary.doc2bow(text) for text in text_data]
import pickle 
pickle.dump(corpus, open('corpus.pkl', 'wb'))

In [14]:
#example
ldamodel=gensim.models.ldamodel.LdaModel(corpus, num_topics=10, id2word=dictionary, passes=6)
for idx, topic in ldamodel.print_topics(-1):
    print('Topic: {} word: {}'.format(idx, topic))

Topic: 0 word: 0.005*"57504",257,"analysis" + 0.005*"37060",170,"and" + 0.005*"50512",225,"lbs" + 0.005*"10673",42,"the" + 0.005*"93988",491,"via" + 0.005*"23790",111,"infrastructurebroadband" + 0.005*"94537",495,"and" + 0.005*"44888",204,"incorporated" + 0.005*"7283",31,"highlights" + 0.005*"51201",228,"to"
Topic: 1 word: 0.005*"90499",463,"co" + 0.005*"44706",204,"management" + 0.005*"64382",285,"the" + 0.005*"10651",42,"the" + 0.005*"12883",59,"of" + 0.005*"11161",45,"state" + 0.005*"35663",164,"property" + 0.005*"71589",328,"montileu" + 0.005*"39919",182,"fy" + 0.005*"78673",371,"date"
Topic: 2 word: 0.006*"73410",340,"transmitters" + 0.006*"23697",111,"other" + 0.006*"90433",462,"commissioners" + 0.006*"29641",137,"refugees" + 0.006*"42866",195,"the" + 0.006*"21870",101,"by" + 0.006*"54643",246,"county" + 0.006*"12546",56,"mandates" + 0.006*"67662",304,"continue" + 0.006*"51677",231,"fy"
Topic: 3 word: 0.006*"2582",16,"for" + 0.006*"17782",82,"caseworkers" + 0.006*"71935",330,"gre

In [15]:
import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

C:\Users\messi\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))
